In [1]:
import pandas as pd
import json
import requests
import math

In [22]:
#Global paramenters
#Name of drug changes
base_url = "https://rxnav.nlm.nih.gov/REST/"
getDrugs = "drugs.json?name="
df = pd.DataFrame()


In [24]:
def query_drug(name_of_drug):
    row = 0
    df = pd.DataFrame()
    rd_df = pd.read_json(base_url+getDrugs+name_of_drug)
    rd_df = pd.DataFrame(rd_df.drugGroup.conceptGroup)
    while row < rd_df.shape[0]:
        if(pd.isna(rd_df.conceptProperties[row]) is True):
            row+=1
        else:
            new_df = pd.DataFrame(rd_df.conceptProperties[row])
            if df.empty:
                df = new_df
            else:
                df = pd.concat([df, new_df]).reset_index(drop=True)
        row += 1
    return df

In [42]:
query_drug('fentanyl')

,rxcui,name,synonym,tty,language,suppress,umlscui
0,1740008,{2 (fentanyl 0.6 MG/ACTUAT Mucosal Spray [Subs...,Subsys 1200 MCG (2 x 600 MCG/ACTUAT) Mucosal S...,BPCK,ENG,N,
1,1740010,{2 (fentanyl 0.8 MG/ACTUAT Mucosal Spray [Subs...,Subsys 1600 MCG (2 x 800 MCG/ACTUAT) Mucosal S...,BPCK,ENG,N,
2,1740007,{2 (fentanyl 0.6 MG/ACTUAT Mucosal Spray) } Pack,fentanyl 1200 MCG (2 x 600 MCG/ACTUAT) Mucosal...,GPCK,ENG,N,
3,1740009,{2 (fentanyl 0.8 MG/ACTUAT Mucosal Spray) } Pack,fentaNYL 1600 MCG (2 x 800 MCG/ACTUAT) Mucosal...,GPCK,ENG,N,
4,1053651,fentanyl 0.1 MG Sublingual Tablet [Abstral],Abstral 0.1 MG Sublingual Tablet,SBD,ENG,N,
...,...,...,...,...,...,...,...
81,858087,fentanyl 1.2 MG Buccal Film,fentanyl 1200 MCG Buccal Film,SCD,ENG,N,
82,858092,fentanyl 0.2 MG Buccal Film,fentanyl (as fentanyl citrate) 200 MCG Buccal ...,SCD,ENG,N,
83,858095,fentanyl 0.4 MG Buccal Film,fentanyl 400 MCG Buccal Film,SCD,ENG,N,
84,858098,fentanyl 0.6 MG Buccal Film,fentanyl 600 MCG Buccal Film,SCD,ENG,N,


In [41]:
#loop array to get complete list
df_opioid = pd.DataFrame()
drugs = ['fentanyl', 'morphine', 'buprenorphine', 'pethidine', 'isonipecaine', 'meperidine', 'dihydromorphinone', 'hydromorphone', 'methadone', 'hydrocodone', 'tapentadol', 'oxycodone', 'oxymorphone', 'codeine']
for drug in drugs:
    merg_df = query_drug(drug)
    df_opioid = pd.concat([df_opioid, merg_df]).drop_duplicates(subset=['rxcui']).reset_index(drop=True)
df_opioid

,rxcui,name,synonym,tty,language,suppress,umlscui
0,1740008,{2 (fentanyl 0.6 MG/ACTUAT Mucosal Spray [Subs...,Subsys 1200 MCG (2 x 600 MCG/ACTUAT) Mucosal S...,BPCK,ENG,N,
1,1740010,{2 (fentanyl 0.8 MG/ACTUAT Mucosal Spray [Subs...,Subsys 1600 MCG (2 x 800 MCG/ACTUAT) Mucosal S...,BPCK,ENG,N,
2,1740007,{2 (fentanyl 0.6 MG/ACTUAT Mucosal Spray) } Pack,fentanyl 1200 MCG (2 x 600 MCG/ACTUAT) Mucosal...,GPCK,ENG,N,
3,1740009,{2 (fentanyl 0.8 MG/ACTUAT Mucosal Spray) } Pack,fentaNYL 1600 MCG (2 x 800 MCG/ACTUAT) Mucosal...,GPCK,ENG,N,
4,1053651,fentanyl 0.1 MG Sublingual Tablet [Abstral],Abstral 0.1 MG Sublingual Tablet,SBD,ENG,N,
...,...,...,...,...,...,...,...
756,997285,codeine phosphate 30 MG Oral Tablet,,SCD,ENG,N,
757,997287,codeine sulfate 30 MG Oral Tablet,,SCD,ENG,N,
758,997296,codeine sulfate 60 MG Oral Tablet,,SCD,ENG,N,
759,997301,codeine phosphate 60 MG Oral Tablet,,SCD,ENG,N,


In [35]:
#Export
df_opioid.to_excel(r"C:/Users/Daniel/Documents/opiod_list/opioid_list/New_Complete_Opioids_List.xlsx", index=False)

In [43]:
df_opioid

,rxcui,name,synonym,tty,language,suppress,umlscui
0,1740008,{2 (fentanyl 0.6 MG/ACTUAT Mucosal Spray [Subs...,Subsys 1200 MCG (2 x 600 MCG/ACTUAT) Mucosal S...,BPCK,ENG,N,
1,1740010,{2 (fentanyl 0.8 MG/ACTUAT Mucosal Spray [Subs...,Subsys 1600 MCG (2 x 800 MCG/ACTUAT) Mucosal S...,BPCK,ENG,N,
2,1740007,{2 (fentanyl 0.6 MG/ACTUAT Mucosal Spray) } Pack,fentanyl 1200 MCG (2 x 600 MCG/ACTUAT) Mucosal...,GPCK,ENG,N,
3,1740009,{2 (fentanyl 0.8 MG/ACTUAT Mucosal Spray) } Pack,fentaNYL 1600 MCG (2 x 800 MCG/ACTUAT) Mucosal...,GPCK,ENG,N,
4,1053651,fentanyl 0.1 MG Sublingual Tablet [Abstral],Abstral 0.1 MG Sublingual Tablet,SBD,ENG,N,
...,...,...,...,...,...,...,...
756,997285,codeine phosphate 30 MG Oral Tablet,,SCD,ENG,N,
757,997287,codeine sulfate 30 MG Oral Tablet,,SCD,ENG,N,
758,997296,codeine sulfate 60 MG Oral Tablet,,SCD,ENG,N,
759,997301,codeine phosphate 60 MG Oral Tablet,,SCD,ENG,N,


In [47]:
#take list and format it to the RXNorm api format

#This list was made by cobing the excel files
df_list = pd.read_excel("C:/Users/Daniel/Documents/opiod_list/opioid_list/Complete_Opioids_List.xlsx")
df_list = df_list[['Code', 'Description']]
df_list.rename(columns= {'Code' : 'rxcui', 'Description': 'name'}, inplace = True)
df_list

,rxcui,name
0,1010600,buprenorphine 2 MG / naloxone 0.5 MG Sublingua...
1,1010604,buprenorphine 8 MG / naloxone 2 MG Sublingual ...
2,1014599,acetaminophen 300 MG / oxycodone hydrochloride...
3,1014615,acetaminophen 300 MG / oxycodone hydrochloride...
4,1014632,acetaminophen 300 MG / oxycodone hydrochloride...
...,...,...
5153,996716,codeine phosphate 20 MG / guaifenesin 400 MG /...
5154,996727,codeine phosphate 20 MG / guaifenesin 400 MG O...
5155,996730,codeine phosphate 20 MG / pseudoephedrine hydr...
5156,996738,codeine phosphate 9 MG / guaifenesin 200 MG Or...


In [48]:
#This list was created by pulling from the RxNorm API
df_opioid = df_opioid[['rxcui', 'name']]
df_opioid

,rxcui,name
0,1740008,{2 (fentanyl 0.6 MG/ACTUAT Mucosal Spray [Subs...
1,1740010,{2 (fentanyl 0.8 MG/ACTUAT Mucosal Spray [Subs...
2,1740007,{2 (fentanyl 0.6 MG/ACTUAT Mucosal Spray) } Pack
3,1740009,{2 (fentanyl 0.8 MG/ACTUAT Mucosal Spray) } Pack
4,1053651,fentanyl 0.1 MG Sublingual Tablet [Abstral]
...,...,...
756,997285,codeine phosphate 30 MG Oral Tablet
757,997287,codeine sulfate 30 MG Oral Tablet
758,997296,codeine sulfate 60 MG Oral Tablet
759,997301,codeine phosphate 60 MG Oral Tablet


In [49]:
same_drug_count = 0
new_drug_count = 0
drug_not_found_count = 0
same_drug = pd.DataFrame({"drugs_in_list_rxcui": [], "drugs_in_list_name": []})
new_drug = pd.DataFrame({"new_drugs_rxcui": [], "new_drugs_name": []})
for code in df_opioid['rxcui']:
    drug_name = df_opioid['name'].where(df_opioid['rxcui'] == code).dropna().values[0]
    if df_list.isin([int(code)]).any().any() == True:
        same_drug_count += 1
        same_drug = same_drug.append({'drugs_in_list_rxcui': code, 'drugs_in_list_name' : drug_name}, ignore_index = True)
    else:
        new_drug_count += 1
        new_drug = new_drug.append({'new_drugs_rxcui': code, 'new_drugs_name': drug_name}, ignore_index = True)
print('Same drug count: ', same_drug_count)
print('New drug count: ', new_drug_count)

Same drug count:  745
New drug count:  16


In [50]:
#Displays same and new drugs
df_final = same_drug.join(new_drug)
df_final.head()

,drugs_in_list_rxcui,drugs_in_list_name,new_drugs_rxcui,new_drugs_name
0,1740008,{2 (fentanyl 0.6 MG/ACTUAT Mucosal Spray [Subs...,2474267,2 ML fentanyl 0.05 MG/ML Prefilled Syringe
1,1740010,{2 (fentanyl 0.8 MG/ACTUAT Mucosal Spray [Subs...,2474269,1 ML fentanyl 0.05 MG/ML Prefilled Syringe
2,1740007,{2 (fentanyl 0.6 MG/ACTUAT Mucosal Spray) } Pack,250304,cyclizine 50 MG/ML / morphine 10 MG/ML Injecta...
3,1740009,{2 (fentanyl 0.8 MG/ACTUAT Mucosal Spray) } Pack,250305,cyclizine 50 MG/ML / morphine 15 MG/ML Injecta...
4,1053651,fentanyl 0.1 MG Sublingual Tablet [Abstral],2539188,1 ML meperidine hydrochloride 50 MG/ML Prefill...


In [51]:
absent_drugs = pd.DataFrame({"drugs_not_found_rxcui": [], "drugs_not_found_name": []})
count = 0
for code in df_list['rxcui']:
    drug_name = df_list['name'].where(df_list['rxcui'] == code).dropna().values[0]
    if df_final.isin([str(code)]).any().any() == True:
        count+=1
    else:
        absent_drugs = absent_drugs.append({'drugs_not_found_rxcui': str(code), 'drugs_not_found_name': drug_name}, ignore_index = True)
print('Count: ', count)
absent_drugs.head()
        
        

Count:  745


,drugs_not_found_rxcui,drugs_not_found_name
0,1113998,chlorpheniramine maleate 0.4 MG/ML / codeine p...
1,1114003,chlorpheniramine maleate 0.4 MG/ML / codeine p...
2,1114110,codeine phosphate 1.6 MG/ML / guaifenesin 40 M...
3,1114878,codeine phosphate 1.6 MG/ML / pseudoephedrine ...
4,1234872,aspirin 356.4 MG / caffeine 30 MG / dihydrocod...


In [52]:
df_final = pd.DataFrame()
df_final = absent_drugs.join(same_drug)
df_final = df_final.join(new_drug)
print(df_final.shape) #check if correct size
df_final.head()

(4413, 6)


,drugs_not_found_rxcui,drugs_not_found_name,drugs_in_list_rxcui,drugs_in_list_name,new_drugs_rxcui,new_drugs_name
0,1113998,chlorpheniramine maleate 0.4 MG/ML / codeine p...,1740008,{2 (fentanyl 0.6 MG/ACTUAT Mucosal Spray [Subs...,2474267,2 ML fentanyl 0.05 MG/ML Prefilled Syringe
1,1114003,chlorpheniramine maleate 0.4 MG/ML / codeine p...,1740010,{2 (fentanyl 0.8 MG/ACTUAT Mucosal Spray [Subs...,2474269,1 ML fentanyl 0.05 MG/ML Prefilled Syringe
2,1114110,codeine phosphate 1.6 MG/ML / guaifenesin 40 M...,1740007,{2 (fentanyl 0.6 MG/ACTUAT Mucosal Spray) } Pack,250304,cyclizine 50 MG/ML / morphine 10 MG/ML Injecta...
3,1114878,codeine phosphate 1.6 MG/ML / pseudoephedrine ...,1740009,{2 (fentanyl 0.8 MG/ACTUAT Mucosal Spray) } Pack,250305,cyclizine 50 MG/ML / morphine 15 MG/ML Injecta...
4,1234872,aspirin 356.4 MG / caffeine 30 MG / dihydrocod...,1053651,fentanyl 0.1 MG Sublingual Tablet [Abstral],2539188,1 ML meperidine hydrochloride 50 MG/ML Prefill...


In [55]:
#export
df_final.to_excel("C:/Users/Daniel/Documents/opiod_list/opioid_list/Opioids_Comparison.xlsx", index=False)

In [53]:
df_final

,drugs_not_found_rxcui,drugs_not_found_name,drugs_in_list_rxcui,drugs_in_list_name,new_drugs_rxcui,new_drugs_name
0,1113998,chlorpheniramine maleate 0.4 MG/ML / codeine p...,1740008,{2 (fentanyl 0.6 MG/ACTUAT Mucosal Spray [Subs...,2474267,2 ML fentanyl 0.05 MG/ML Prefilled Syringe
1,1114003,chlorpheniramine maleate 0.4 MG/ML / codeine p...,1740010,{2 (fentanyl 0.8 MG/ACTUAT Mucosal Spray [Subs...,2474269,1 ML fentanyl 0.05 MG/ML Prefilled Syringe
2,1114110,codeine phosphate 1.6 MG/ML / guaifenesin 40 M...,1740007,{2 (fentanyl 0.6 MG/ACTUAT Mucosal Spray) } Pack,250304,cyclizine 50 MG/ML / morphine 10 MG/ML Injecta...
3,1114878,codeine phosphate 1.6 MG/ML / pseudoephedrine ...,1740009,{2 (fentanyl 0.8 MG/ACTUAT Mucosal Spray) } Pack,250305,cyclizine 50 MG/ML / morphine 15 MG/ML Injecta...
4,1234872,aspirin 356.4 MG / caffeine 30 MG / dihydrocod...,1053651,fentanyl 0.1 MG Sublingual Tablet [Abstral],2539188,1 ML meperidine hydrochloride 50 MG/ML Prefill...
...,...,...,...,...,...,...
4408,860530,guaifenesin 20 MG/ML / hydrocodone bitartrate ...,NaN,NaN,NaN,NaN
4409,891172,guaifenesin 20 MG/ML / hydrocodone bitartrate ...,NaN,NaN,NaN,NaN
4410,996462,codeine phosphate 2 MG/ML / guaifenesin 20 MG/...,NaN,NaN,NaN,NaN
4411,996484,codeine phosphate 2 MG/ML / guaifenesin 20 MG/...,NaN,NaN,NaN,NaN


In [310]:
#start here for absolete
#reduce data to be used to find obsolete
not_found = df_final[['drugs_not_found_rxcui', 'drugs_not_found_name']]
not_found.rename(columns= {'drugs_not_found_rxcui' : 'rxcui', 'drugs_not_found_name': 'name'}, inplace = True)
not_found

/Users/hm/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,rxcui,name
0,1010600,buprenorphine 2 MG / naloxone 0.5 MG Sublingua...
1,1010604,buprenorphine 8 MG / naloxone 2 MG Sublingual ...
2,1014599,acetaminophen 300 MG / oxycodone hydrochloride...
3,1014615,acetaminophen 300 MG / oxycodone hydrochloride...
4,1014632,acetaminophen 300 MG / oxycodone hydrochloride...
...,...,...
4962,996716,codeine phosphate 20 MG / guaifenesin 400 MG /...
4963,996727,codeine phosphate 20 MG / guaifenesin 400 MG O...
4964,996730,codeine phosphate 20 MG / pseudoephedrine hydr...
4965,996738,codeine phosphate 9 MG / guaifenesin 200 MG Or...


In [315]:
df_status = pd.DataFrame()
for code in not_found['rxcui']:
    getRxcuiHistoryStatus = 'rxcui/{}/historystatus.json'.format(code)
    df_type = pd.read_json(base_url+getRxcuiHistoryStatus)
    df_type = pd.DataFrame(df_type.rxcuiStatusHistory.metaData, index=[0])
    df_type['rxcui'] = code
    if(df_type.shape[0] != 1):
        print('error')
    df_status = df_status.append(df_type)

KeyboardInterrupt: 

In [316]:
df_status

,status,source,releaseStartDate,releaseEndDate,isCurrent,activeStartDate,activeEndDate,remappedDate,rxcui
0,Active,RXNORM,102010,,YES,102010,,,1010600
0,Active,RXNORM,102010,,YES,102010,,,1010604
0,Active,RXNORM,112010,,YES,112010,,,1014599
0,Active,RXNORM,112010,,YES,112010,,,1014615
0,Active,RXNORM,112010,,YES,112010,,,1014632
...,...,...,...,...,...,...,...,...,...
0,Active,RXNORM,082014,,YES,082014,,,1542997
0,Active,RXNORM,092014,,YES,092014,,,1544851
0,Active,RXNORM,092014,,YES,092014,,,1544854
0,Active,RXNORM,012015,,YES,012015,,,1596108


In [317]:
df_not_found = not_found.merge(df_status, on='rxcui')

In [318]:
df_not_found.head()

,rxcui,name,status,source,releaseStartDate,releaseEndDate,isCurrent,activeStartDate,activeEndDate,remappedDate
0,1010600,buprenorphine 2 MG / naloxone 0.5 MG Sublingua...,Active,RXNORM,102010,,YES,102010,,
1,1010604,buprenorphine 8 MG / naloxone 2 MG Sublingual ...,Active,RXNORM,102010,,YES,102010,,
2,1014599,acetaminophen 300 MG / oxycodone hydrochloride...,Active,RXNORM,112010,,YES,112010,,
3,1014615,acetaminophen 300 MG / oxycodone hydrochloride...,Active,RXNORM,112010,,YES,112010,,
4,1014632,acetaminophen 300 MG / oxycodone hydrochloride...,Active,RXNORM,112010,,YES,112010,,


In [319]:
#export
df_not_found.to_excel(r"/Users/hm/Documents/opioid_list/opioid_list/Status_of_Not_found.xlsx", index=False)